In [ ]:
from dotenv import load_dotenv

from computer_use_demo.tools import ComputerTool, ToolCollection

load_dotenv()

import asyncio

from computer_use_demo.loop import completion, PROVIDER_TO_DEFAULT_MODEL_NAME, APIProvider, SYSTEM_PROMPT
import os

from playwright.async_api import async_playwright
from termcolor import colored

In [ ]:
def prefixed_print(content, prefix=""):
    print(*[prefix + line for line in content.split("\n")], sep="\n")

def print_conversation():
    for message in messages:
            role = message.get("role")
            content = message.get("content")
            
            on_color = "on_green" if role == "user" else "on_red"
            color = "green" if role == "user" else "red"
            print(colored(f"[{role}]:", "black", on_color, ["bold"]))
            
            def print_content(content, prefix=""):
                if isinstance(content, str):
                    print(colored(content, color))
                    return
                
                for content_part in content:
                    if isinstance(content_part, str):
                        print(colored(content_part, color))
                        continue
                    c_type = content_part.get("type")
                    if c_type == "text":
                        prefixed_print(colored(content_part.get("text"), color), prefix)
                    elif c_type == "image":
                        prefixed_print(colored("Image", "yellow"), prefix)
                        print_content([content_part.get("source")], prefix + "  ")
                    elif c_type == "base64":
                        prefixed_print(colored(f"[{content_part.get("media_type")}]: <{content_part.get("data")[:10]}...>", "yellow"), prefix)
                    elif c_type == "tool_use":
                        prefixed_print(colored(f"Using tool {content_part.get("name")} with input {content_part.get("input")}", "blue"), prefix)
                    elif c_type == "tool_result":
                        prefixed_print(colored(f"Tool result:", "blue"), prefix)
                        print_content(content_part.get("content"), prefix + "  ")
            
            print_content(content)  
            print("\n")
            
def print_stuff(prefix, *args, **kwargs):
    # pass
    print(f"[{prefix}]", args, kwargs)
    
def noop(*_, **__):
    pass


In [ ]:
messages = []

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page(viewport={"width": 1024, "height": 768})

In [ ]:
# import base64
# 
# await page.goto("https://stackoverflow.com")
# screenshot = await page.screenshot(timeout=0)
# screenshot_base64 = base64.b64encode(screenshot).decode("utf-8")
# prompt = {
#     "role": "user",
#     "content": [
#         {
#             "type": "text",
#             "text": "Given the following page, perform all the necessary actions to dismiss/close any popups/dialogs you see. Do not take a screenshot, work with the provided page."
#         },
#         {
#             "type": "image",
#             "source": {
#                 "type": "base64",
#                 "media_type": "image/png",
#                 "data": screenshot_base64,
#             },
#         }
#     ]
# }
# messages.append(prompt)

In [ ]:
import base64

await page.goto("https://www.bidnetdirect.com/colorado/cityandcountyofdenverdepartmentofaviation")
screenshot = await page.screenshot(timeout=0)
screenshot_base64 = base64.b64encode(screenshot).decode("utf-8")
prompt = {
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Given the following page, go to \"Closed contract\" tab. Do not ask for a screenshot, work with the provided screenshot."
        },
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": screenshot_base64,
            },
        }
    ]
}
messages = [prompt]

In [ ]:
# import base64
# 
# await page.goto("https://smctd.bonfirehub.com/portal/?tab=openOpportunities")
# screenshot = await page.screenshot(timeout=0)
# screenshot_base64 = base64.b64encode(screenshot).decode("utf-8")
# prompt = {
#     "role": "user",
#     "content": [
#         {
#             "type": "text",
#             "text": "Given the following page, go to the details page of the first contract you see. Do not ask for a screenshot, work with the provided screenshot."
#         },
#         {
#             "type": "image",
#             "source": {
#                 "type": "base64",
#                 "media_type": "image/png",
#                 "data": screenshot_base64,
#             },
#         }
#     ]
# }
# messages = [prompt]

In [ ]:
tool_collection = ToolCollection(ComputerTool(page))
response = await completion(
    model=PROVIDER_TO_DEFAULT_MODEL_NAME[APIProvider.ANTHROPIC],
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
    provider=APIProvider.ANTHROPIC,
    messages=messages,
    api_response_callback=noop,
    tool_collection=tool_collection,
)
messages.append({
    "role": "assistant",
    "content": response,
})
print_conversation()

In [ ]:
from typing import Any, cast

tool_result_content = []
for content_block in response:
    if content_block["type"] == "tool_use":
        result = await tool_collection.run(
            name=content_block["name"],
            tool_input=cast(dict[str, Any], content_block["input"]),
        )
        tool_result_content.append(
            (result, content_block["id"])
        )
tool_result_content